#### Adicionando os paths

In [1]:
import sys
sys.path.append('../')
sys.path.append('../Scripts/')

#### Importando bibliotecas

In [2]:
import pandas as pd
from Global_Variables import *
import Aux_Functions as af

In [3]:
atributos = pd.read_csv("../Outputs/Tabela_Atributos.csv", sep=";")

In [4]:
atributos

,Sequence,Genotype,Position,Codon,Drug,Codon_Resistente_01,Codon_Resistente_02,Codon_Resistente_03,Codon_Resistente_04,Codon_Resistente_05,...,Codon_Usage_09,Barreira_Genetica_10,Prevalencia_10,Codon_Usage_10,Barreira_Genetica_11,Prevalencia_11,Codon_Usage_11,Barreira_Genetica_12,Prevalencia_12,Codon_Usage_12
0,JX112598.1a.US.2009,1a,24,AAG,DCV,AGA,AGG,CGA,CGC,CGG,...,0,-1,0,0,-1,0,0,-1,0,0
1,JX112598.1a.US.2009,1a,24,AAG,LDV,AGA,AGG,CGA,CGC,CGG,...,0,-1,0,0,-1,0,0,-1,0,0
2,JX112598.1a.US.2009,1a,28,ATG,DCV,GTA,GTC,GTG,GTT,-,...,0,-1,0,0,-1,0,0,-1,0,0
3,JX112598.1a.US.2009,1a,28,ATG,OMV,GTA,GTC,GTG,GTT,-,...,0,-1,0,0,-1,0,0,-1,0,0
4,JX112598.1a.US.2009,1a,28,ATG,PIB,GTA,GTC,GTG,GTT,-,...,0,-1,0,0,-1,0,0,-1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36985,FJ958693.1a.US.2005,1a,93,TAC,EBR,CAC,CAT,-,-,-,...,0,-1,0,0,-1,0,0,-1,0,0
36986,FJ958693.1a.US.2005,1a,93,TAC,LDV,CAC,CAT,-,-,-,...,0,-1,0,0,-1,0,0,-1,0,0
36987,FJ958693.1a.US.2005,1a,93,TAC,OMV,CAC,CAT,-,-,-,...,0,-1,0,0,-1,0,0,-1,0,0
36988,FJ958693.1a.US.2005,1a,93,TAC,PIB,CAC,CAT,-,-,-,...,0,-1,0,0,-1,0,0,-1,0,0


In [5]:
atributos.iloc[:,17:]

,Barreira_Genetica_01,Prevalencia_01,Codon_Usage_01,Barreira_Genetica_02,Prevalencia_02,Codon_Usage_02,Barreira_Genetica_03,Prevalencia_03,Codon_Usage_03,Barreira_Genetica_04,...,Codon_Usage_09,Barreira_Genetica_10,Prevalencia_10,Codon_Usage_10,Barreira_Genetica_11,Prevalencia_11,Codon_Usage_11,Barreira_Genetica_12,Prevalencia_12,Codon_Usage_12
0,2.0,0,0,1.0,0,0,4.5,0,0,6.0,...,0,-1,0,0,-1,0,0,-1,0,0
1,2.0,0,0,1.0,0,0,4.5,0,0,6.0,...,0,-1,0,0,-1,0,0,-1,0,0
2,2.0,0,0,3.5,0,0,1.0,0,0,3.5,...,0,-1,0,0,-1,0,0,-1,0,0
3,2.0,0,0,3.5,0,0,1.0,0,0,3.5,...,0,-1,0,0,-1,0,0,-1,0,0
4,2.0,0,0,3.5,0,0,1.0,0,0,3.5,...,0,-1,0,0,-1,0,0,-1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36985,1.0,0,0,2.0,0,0,-1.0,0,0,-1.0,...,0,-1,0,0,-1,0,0,-1,0,0
36986,1.0,0,0,2.0,0,0,-1.0,0,0,-1.0,...,0,-1,0,0,-1,0,0,-1,0,0
36987,1.0,0,0,2.0,0,0,-1.0,0,0,-1.0,...,0,-1,0,0,-1,0,0,-1,0,0
36988,1.0,0,0,2.0,0,0,-1.0,0,0,-1.0,...,0,-1,0,0,-1,0,0,-1,0,0
